# H&M - Collaborative Filtering: User-user
![](https://storage.googleapis.com/kaggle-competitions/kaggle/31254/logos/header.png)

## A user-user collaborative filtering (UUCF) model for the competition [H&M Personalized Fashion Recommendations](https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations).


A UUCF (user-user collaborative filtering) is one of the most common Recommender System. The core idea behind it is as follow: 
* For a given user (`u`), we will search for similar users. Let's say, the 10 most similar users to `u`. This is `u`'s neighborhood. 
* We search for the most popular items in this neighborhood of users similar to `u`, `I`
* We check the items that are popular in the group of users similar to `u`, that `u` hasn't purchased yet.
* We recommend those items to `u`

Now, we need to operationalize two things here: 
* What does it mean that 2 users are similar?
* What does it mean that an item is popular in a group of users?

For the similarity among users, we will just compare their purchases. The overlap of their purchased items is the similarity of the users. With this idea, we want to capture users with similar consumption patterns.

For the popularity of items in a group, there are more simple and more complex definitions. We can just take the number of purchases a given product has in a group and rank all the possible products with that metric. Or we can consider the similarity to the original user `u` and weight the purchases depending on who was purchasing. That way, purchases of users that are more similar to `u` will have a stronger impact on the overall ranking of products.

---

**In version 20 I have removed Heng's "time is our best friend" model as the fallback model for cold start and put a simple 12-most popular one in its place. Heng's baseline seems to be performing much better than my UUCF so the reporting a `0.019` score is actually a `-0.001` against doing nothing. I want to see the result of the UUCF by itself. I was expecting to outperform the baseline with a subset of users under the UUCF regime, but I couldn't till now. The next steps in order to make UUCF work would be to have a validation strategy and extract the hyperparameters of the code in order to find the optimal combination. Hope someone can make the UUCF perform better (I know it can).**

If I can obtain a score that surpasses the `0.02`, using UUCF + Heng's baseline, I will share it for sure.



# Please, _DO_ upvote if you find this kernel useful or interesting!

# Imports and prepare data

In [1]:
import time
import numpy as np
import pandas as pd

import multiprocessing as mp
from multiprocessing import Pool
from functools import partial

In [2]:
base_path = '../input/h-and-m-personalized-fashion-recommendations/'
csv_train = f'{base_path}transactions_train.csv'
csv_sub = f'{base_path}sample_submission.csv'
csv_users = f'{base_path}customers.csv'
csv_items = f'{base_path}articles.csv'

df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])
df_sub = pd.read_csv(csv_sub)

In [3]:
df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])

In [4]:
# s0 = (df['t_dat'] >= '2019-09-15')&(df['t_dat'] < '2019-09-30')
s1 = (df['t_dat'] < '2020-09-15')


In [5]:
df = df[s1]

# Create mapping from ids to incremental integers and viceversa

* We will reserve the term `user_id` as a integer from 0 and `customer_id` for the original id.
* The same goes with `item_id` and `article_id`

In [6]:
dfu = pd.read_csv(csv_users)
dfi = pd.read_csv(csv_items, dtype={'article_id': str})

ALL_USERS = dfu['customer_id'].unique().tolist()
ALL_ITEMS = dfi['article_id'].unique().tolist()

user_to_customer_map = {user_id: customer_id for user_id, customer_id in enumerate(ALL_USERS)}
customer_to_user_map = {customer_id: user_id for user_id, customer_id in enumerate(ALL_USERS)}

item_to_article_map = {item_id: article_id for item_id, article_id in enumerate(ALL_ITEMS)}
article_to_item_map = {article_id: item_id for item_id, article_id in enumerate(ALL_ITEMS)}

del dfu, dfi

In [7]:
df['user_id'] = df['customer_id'].map(customer_to_user_map)
df['item_id'] = df['article_id'].map(article_to_item_map)

# Build model

# Configuration parameters

Since UUCF is very computationally expensive, we will apply it only to a small subset of users. The hope is that, for those users, the recommendation are better than the other models.

We will reduce the data from 2 fronts:
* Keep only the most recent history. That is `START_DATE`.
* Keep only users with at least `MINIMUM_PURCHASES`

In [8]:
N_SIMILAR_USERS = 30

MINIMUM_PURCHASES = 2

#START_DATE = '2020-08-21'
START_DATE = '2020-08-14' # for 104

#START_DATE = '2020-09-01'
#START_DATE = '2018-09-01'


DROP_PURCHASED_ITEMS = False

DROP_USER_FROM_HIS_NEIGHBORHOOD = False

TEST_RUN = False

TEST_SIZE = 1000

In [9]:
def flatten(l):
    """ Flatten a list of lists"""
    return [item for sublist in l for item in sublist]

def compare_vectors(v1, v2):
    """Compare lists of purchased product for two given users
    v1 stands for the "vector representation for user 1", which is a list of the purchases of u1
    
    Returns:
        A value between 0 and 1 (similarity)
    """
    intersection = len(set(v1) & set(v2))
    denominator = np.sqrt(len(v1) * len(v2))
    return intersection / denominator

def get_similar_users(u, v, dfh):
    """
    Get the N_SIMILAR_USERS most similar users to the given one with their similarity score
    Arguments:
        u: the user_id, 
        v:  the "vector" representation of the user (list of item_id)
        dfh : the "history of transaccions" dataframe
        
    Returns:
        tuple of lists ([similar user_id], [similarity scores])
    """
    similar_users = dfh.apply(lambda v_other: compare_vectors(v, v_other)).sort_values(ascending=False).head(N_SIMILAR_USERS + 1)
    
    if DROP_USER_FROM_HIS_NEIGHBORHOOD:
        similar_users = similar_users[similar_users.index != u]
        
    return similar_users.index.tolist(), similar_users.tolist()

def get_items(u, v, dfh):
    """ Get the recommend items for a given users
    
    It will:
        1) Get similar users for the given user
        2) Obtain all the items those users purchased
        3) Rank them using the similarity scores of the user that purchased them
        4) Return the 12 best ranked
    
    Arguments:
        u: the user_id, 
        v:  the "vector" representation of the user (list of item_id)
        dfh : the "history of transaccions" dataframe
        
    Returns:
        list of item_id of lenght at most 12
    """
    global i, n
    
    users, scores = get_similar_users(u, v, dfh)
    df_nn = pd.DataFrame({'user': users, 'score': scores})
    df_nn['items'] = df_nn.apply(lambda row: dfh.loc[row.user], axis=1)
    df_nn['weighted_items'] = df_nn.apply(lambda row: [(item, row.score) for item in row['items']], axis=1)

    recs = pd.DataFrame(flatten(df_nn['weighted_items'].tolist()), columns=['item', 'score']).groupby('item')['score'].sum().sort_values(ascending=False)
    if DROP_PURCHASED_ITEMS:
        recs = recs[~recs.index.isin(v)]
    # Keep the first 12 and get the item_ids
    i +=1
    if i % 1000 == 0:
        pid = mp.current_process().pid
        print(f"[PID {pid:>2d}] Finished {i:3d} / {n:5d} - {i/n*100:3.0f}%")
    return recs.head(12).index.tolist()

def get_items_chunk(user_ids: np.array, dfh: pd.DataFrame):
    """ Call get_item for a list of user_ids
    
    Arguments:
        user_ids: list of user_id, 
        dfh: the "history of transaccions" dataframe
        
    Returns:
        pd.Series with index user_id and list of item_id (recommendations) as value
    """
    global i, n
    i = 0
    
    n = len(user_ids)
    pid = mp.current_process().pid
    print(f"[PID {pid:>2d}] Started working with {n:5d} users")
    
    df_user_vectors = pd.DataFrame(dfh.loc[user_ids]).reset_index()
    df_user_vectors['recs'] = df_user_vectors.apply(lambda row: get_items(row.user_id, row.item_id, dfh), axis=1)
    return df_user_vectors.set_index('user_id')['recs']

def get_recommendations(users: list, dfh: pd.DataFrame):
    """
    Obtained recommendation for the users using transaccion dfh in a parallelized manner
    
    Call get_items_chunk in a "smart" multiprocessing fashion
    
    Arguments:
        users: list of user_id
        dfh: the "history of transaccions" dataframe
    
    Returns:
        pd.DataFrame with index user_id and list of item_id (recommendations) as value
    
    """
    time_start = time.time()
    
    # Split into approximately evenly sized chunks
    # We will send just one batch to each CPU 
    user_chunks = np.array_split(users, mp.cpu_count())
    
    f = partial(get_items_chunk, dfh=dfh)
    with Pool(mp.cpu_count()) as p:
        res = p.map(f, user_chunks)
    
    df_rec = pd.DataFrame(pd.concat(res))

    elapsed = (time.time() - time_start) / 60
    print(f"Finished get_recommendations({len(users)}). It took {elapsed:5.2f} mins")
    return df_rec


def uucf(df, start_date=START_DATE):
    """ Entry point for the UUCF model. 
    
    Receive the original transactions_train.csv and a start_date and gets UUCF recommendations
    
    The model will not cover the full list of users, but just a subset of them.
    
    It will provide recommendations for users with at least MINIMUM_PURCHASES after start_date.
    It might return less than 12 recs per user.
    
    An ad-hoc function for filling these gaps should be used downstream.
    (See fill functionality right below)
    
    
    Arguments:
        df: The raw dataframe from transactions_train.csv
        start_date: a date
        
    Returns:
        a submission-like pd.DataFrame with columns [customer_id, prediction]
        'prediction' is a list and not a string though
    
    """
    df_small = df[df['t_dat'] > start_date]
    print(f"Kept data from {start_date} on. Total rows: {len(df_small)}")
    
    # H stands for "Transaction history"
    # dfh is a series of user_id => list of item_id (the list of purchases in order)
    dfh = df_small.groupby("user_id")['item_id'].apply(lambda items: list(set(items)))
    dfh = dfh[dfh.str.len() >= MINIMUM_PURCHASES]
    if TEST_RUN:
        print("WARNING: TEST_RUN is True. It will be a toy execution.")
        dfh = dfh.head(TEST_SIZE)
    
    users = dfh.index.tolist()
    n_users = len(users)
    print(f"Total users in the time frame with at least {MINIMUM_PURCHASES}: {n_users}")
    
    df_rec = get_recommendations(users, dfh)
    df_rec['customer_id'] = df_rec.index.map(user_to_customer_map)
    df_rec['prediction'] = df_rec['recs'].map(lambda l: [item_to_article_map[i] for i in l])
    
    # Submission ready dataframe
    df_rec.reset_index(drop=True)[['customer_id', 'prediction']]
    return df_rec 

In [10]:
df_recs = uucf(df)

Kept data from 2020-08-14 on. Total rows: 1153155
Total users in the time frame with at least 2: 190525
[PID 5961] Started working with 31755 users
[PID 5962] Started working with 31754 users
[PID 5963] Started working with 31754 users
[PID 5964] Started working with 31754 users
[PID 5965] Started working with 31754 users
[PID 5966] Started working with 31754 users
[PID 5962] Finished 1000 / 31754 -   3%
[PID 5963] Finished 1000 / 31754 -   3%
[PID 5964] Finished 1000 / 31754 -   3%
[PID 5965] Finished 1000 / 31754 -   3%
[PID 5961] Finished 1000 / 31755 -   3%
[PID 5966] Finished 1000 / 31754 -   3%
[PID 5962] Finished 2000 / 31754 -   6%
[PID 5964] Finished 2000 / 31754 -   6%
[PID 5963] Finished 2000 / 31754 -   6%
[PID 5961] Finished 2000 / 31755 -   6%
[PID 5965] Finished 2000 / 31754 -   6%
[PID 5966] Finished 2000 / 31754 -   6%
[PID 5962] Finished 3000 / 31754 -   9%
[PID 5964] Finished 3000 / 31754 -   9%
[PID 5963] Finished 3000 / 31754 -   9%
[PID 5961] Finished 3000 / 31755

# Fill the remainder with another model
We will use [Heng Zheng](https://www.kaggle.com/hengzheng)'s [time is our best friend v2](https://www.kaggle.com/hengzheng/time-is-our-best-friend-v2/) which is simple and the best performing public model as of today.

I have created a dataset with the submission file from that notebook [here](https://www.kaggle.com/julian3833/heng-zhengs-time-is-our-best-friend-v2-submission).

In [11]:
csv_fill = csv_sub#'../input/h-m-content-based-12-most-popular-items-0-007/submission.csv'
df_fill = pd.read_csv(csv_fill)

In [12]:
def drop_duplicates(seq):
    """ Remove duplicates of a given sequence keeping order"""
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def fill_row(row):
    uucf = row['prediction_uucf']
    fill = row['prediction_fill'].split()
    new_list = drop_duplicates(uucf + fill)[:12]
    return ' '.join(new_list)


def fill(df_recs, df_fill):
    df_recs['len'] = df_recs['prediction'].str.len()
    df_recs = pd.merge(df_fill, df_recs, how='left', on='customer_id', suffixes=('_fill', '_uucf'))
    
    
    # No recs from UUCF at all: use the fallback model 
    df_recs.loc[df_recs['prediction_uucf'].isnull(), 'prediction'] = df_recs['prediction_fill']


    # Full UUCF recommendation
    mask = df_recs['prediction_uucf'].notnull() & (df_recs['len'] == 12)
    df_recs.loc[mask, 'prediction'] = df_recs['prediction_uucf']


    # Fill with another model. Not enough recs from UUCF
    fill_mask = df_recs['prediction_uucf'].notnull() & (df_recs['len'] < 12)
    df_recs.loc[fill_mask, 'prediction'] = df_recs[fill_mask].apply(fill_row, axis=1)
    return df_recs.drop(['prediction_uucf', 'prediction_fill', 'len', 'recs'], axis=1)

In [13]:
df_recs

,recs,customer_id,prediction
user_id,,,
6,"[3091, 58295, 3094, 3088, 2252, 88300, 95385, ...",0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,"[0448509014, 0719530003, 0448509018, 044850900..."
13,"[15989, 7725, 60792, 60805, 100322, 16004, 142...",00009d946eec3ea54add5ba56d5210ea898def4b46c685...,"[0568597007, 0516859008, 0730863005, 073086303..."
15,"[53914, 53902, 103669, 53892, 53893, 53894, 82...",0000b2f1829e23b24feec422ef13df3ccedaedc85368e6...,"[0706016038, 0706016015, 0914441005, 070601600..."
29,"[104045, 95950, 96063, 104047, 77915, 93020, 2...",00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d...,"[0918292001, 0868405004, 0868823008, 091829200..."
36,"[77908, 73698, 77915, 73703, 185, 58501, 62638...",0001b0127d3e5ff8dadcfc6e5043682dba2070f2667081...,"[0791587001, 0772902001, 0791587015, 077290201..."
...,...,...,...
1371960,"[88821, 76731, 94700, 54341, 97246, 103304, 10...",fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,"[0834906012, 0786187002, 0863646005, 070726900..."
1371963,"[96272, 96812, 96270, 67, 98937, 96676, 98152,...",ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,"[0869706005, 0871664001, 0869706001, 015623100..."
1371969,"[91131, 82965, 100271, 6917, 103007, 69721, 40...",ffff61677073258d461e043cc9ed4ed97be5617a920640...,"[0846782002, 0810746001, 0889392001, 050909102..."


In [14]:
# Fill with another model
df_sub = fill(df_recs, df_fill)
df_sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [15]:
df_sub.shape

(1371980, 2)

In [16]:
# Submit
df_sub.to_csv("uucf104-2-0814.csv", index=False)

# Please, _DO_ upvote if you find this kernel useful or interesting!